In [3]:
import cv2
import mediapipe as mp
import csv
from datetime import datetime, timedelta
import math
import numpy as np


# #CSV 파일에 헤더를 추가
with open('test.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['r','g','b', 'Mouth Openness','mouth_width', 'Label'])


# Mediapipe의 Face Mesh 모델 로드
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

# 웹캠에서 영상을 받아오는 VideoCapture 객체 생성
cap = cv2.VideoCapture(0)

start_time = datetime.now()
elapsed_time = timedelta(seconds=0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("비디오 캡처 실패, 종료합니다.")
        break

    frame = cv2.flip(frame, 1)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # 프레임을 BGR에서 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # 얼굴 검출 및 랜드마크 추출
    results = face_mesh.process(rgb_frame)

    
    if results.multi_face_landmarks:
     
        for face_landmarks in results.multi_face_landmarks:

            mouth_area=frame.copy()

            landmark_indices =[0,61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291,37,39,40,267,269,270]
         


            landmarks=[face_landmarks.landmark[i]for i in landmark_indices]
          
            
         
            h, w, _ = frame.shape
            x_1 = int(np.mean([landmark.x * w for landmark in landmarks]))
            y_1 = int(np.mean([landmark.y * h for landmark in landmarks]))-5


            cv2.circle(frame, (x_1, y_1),2, (0, 255, 0), -1)
            
            
        

            for idx in landmark_indices:
                landmark = face_landmarks.landmark[idx]
                x = int(landmark.x * frame.shape[1])
                y = int(landmark.y * frame.shape[0])
                cv2.circle(frame,(x,y),3,(255,0,0),-1)
            

            # rgb값과 brightness값
            mask = np.zeros(gray_frame.shape, dtype=np.uint8)
            points = np.array([[int(landmark.x * w), int(landmark.y * h)] for landmark in landmarks], dtype=np.int32)
            cv2.fillPoly(mask, [points], 255)

            brightness = cv2.mean(gray_frame, mask=mask)[0]
            # print(brightness)
  
          


            if y_1 < h and x_1 < w:
                rgb_value = rgb_frame[y_1, x_1]
                r, g, b = rgb_value[0], rgb_value[1], rgb_value[2]
                r=round(r/brightness,4)
                g=round(g/brightness,4)
                b=round(b/brightness,4)

            else:
                print(f"Invalid coordinates: face not detected")

           
                   
            #웹캠과의 거리에 따라 입의 가로길이와 세로길이가 변경함에 따라 기준값으로 얼굴너비를 설정

            face_width = math.sqrt(
            (face_landmarks.landmark[172].x - face_landmarks.landmark[264].x) ** 2 +
            (face_landmarks.landmark[172].y - face_landmarks.landmark[264].y) ** 2
        )
            face_width=20*face_width
        

            #입의 세로길이=>고중저모음 판단기준

            upper_lip_bottom = (face_landmarks.landmark[12].x, face_landmarks.landmark[12].y)
            lower_lip_top = (face_landmarks.landmark[14].x, face_landmarks.landmark[14].y)
            mouth_openness = (lower_lip_top[1] - upper_lip_bottom[1])/face_width
            mouth_openness=round(100*mouth_openness,4)

        
            # 입의 가로길이를 계산=>원순/평순 판단기준
            
            lip_left =(face_landmarks.landmark[61].x, face_landmarks.landmark[61].y)
            lip_right=(face_landmarks.landmark[291].x, face_landmarks.landmark[291].y)
            mouth_width=(lip_right[0]-lip_left[0])/face_width
            mouth_width=round(10*mouth_width,4)

          
            # 경과 시간이 0.2초 이상일 때만 CSV 파일에 추가
            if elapsed_time.total_seconds() >= 0.2:
                # 시간 단위로 입안 명암도, 입 벌림 정도, 입의 가로길이를 CSV 파일에 추가

                with open('test.csv', mode='a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([r,g,b,mouth_openness,mouth_width,]) 
                    

                # 경과 시간 초기화
                elapsed_time = timedelta(seconds=0)
            else:
                # 경과 시간 누적
                elapsed_time += datetime.now() - start_time
                start_time = datetime.now()
            

            # 입 벌림 정도, 입안 명암도, 입의 가로길이를 프레임에 표시
           
            cv2.putText(frame, f"Mouth Openness: {mouth_openness:.4f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, f"Mouth Width: {mouth_width:.4f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            cv2.putText(frame, f"RGB: ({r:},{g:},{b:})", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
       
            
    # 영상 출력
    cv2.imshow('Face Mesh', frame)
    
    # 종료 키 (q) 입력 시 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 


# 자원 해제
cap.release()
cv2.destroyAllWindows()


In [4]:
import tensorflow as tf
import joblib
import pandas as pd
import numpy as np
from collections import Counter
import firebase_admin
from firebase_admin import credentials, db

# 모델과 스케일러 로드
model1 = tf.keras.models.load_model('722.keras')
scaler1 = joblib.load('scaler2.joblib')


# 새로운 데이터 로드 및 전처리
new_data = pd.read_csv('test.csv')
new_data = np.array(new_data)
scaled_data = scaler1.transform(new_data[:,:5]) # transform만 수행

# 예측 수행
predictions = model1.predict(scaled_data)

# 예측 결과 출력
predicted_classes = []
for i, prediction in enumerate(predictions):
    predicted_class = np.argmax(prediction)
    print(f"새로운 데이터 {i+1}의 예측 클래스: {predicted_class}")
    predicted_classes.append(predicted_class)

most_common_class = Counter(predicted_classes).most_common(1)[0][0]
print(f"예측 클래스들의 최빈값: {most_common_class}")

# Firebase 초기화
# cred = credentials.Certificate("firebasepy.json")
# firebase_admin.initialize_app(cred, {
#     'databaseURL': 'https://anproject-846d0-default-rtdb.firebaseio.com/'
# })

def get_next_path(base_path):
    ref = db.reference(base_path)
    snapshot = ref.get()
    if isinstance(snapshot, dict):
        keys = list(snapshot.keys())
        next_index = len(keys) + 1
    elif isinstance(snapshot, list):
        next_index = len(snapshot)
    else:
        next_index = 1
    return f"{base_path}/{next_index}"

# 데이터 업로드
base_path = 'devices/Result'
next_path = get_next_path(base_path)
common_ref = db.reference(next_path)
common_ref.set({'vowel': int(most_common_class)})

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
새로운 데이터 1의 예측 클래스: 6
새로운 데이터 2의 예측 클래스: 3
새로운 데이터 3의 예측 클래스: 6
새로운 데이터 4의 예측 클래스: 6
새로운 데이터 5의 예측 클래스: 4
새로운 데이터 6의 예측 클래스: 3
새로운 데이터 7의 예측 클래스: 4
새로운 데이터 8의 예측 클래스: 3
새로운 데이터 9의 예측 클래스: 3
새로운 데이터 10의 예측 클래스: 3
새로운 데이터 11의 예측 클래스: 3
새로운 데이터 12의 예측 클래스: 3
새로운 데이터 13의 예측 클래스: 3
새로운 데이터 14의 예측 클래스: 3
새로운 데이터 15의 예측 클래스: 3
예측 클래스들의 최빈값: 3
